In [ ]:
from google.colab import drive
drive.mount("/content/drive")

!pip install xlrd
!pip install openpyxl
!pip install finance-datareader
!pip install transformers

import pandas as pd
import numpy as np
import FinanceDataReader as fdr
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


import random
import time
import datetime

Mounted at /content/drive
     |████████████████████████████████| 3.1 MB 5.2 MB/s 
     |████████████████████████████████| 59 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 53.8 MB/s 
     |████████████████████████████████| 895 kB 53.0 MB/s 
     |████████████████████████████████| 3.3 MB 46.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
company1 = pd.read_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/[종목크롤링]/코스피_토큰화_불용어제거_v01.csv', encoding = 'utf-8')
company2 = pd.read_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/[종목크롤링]/코스닥_토큰화_불용어제거_v01.csv', encoding = 'utf-8')
issue = pd.read_excel('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/최종이슈/찐최종이슈_토픽모델링_단어 (2).xlsx')
issue_date = pd.read_excel('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/최종이슈/찐최종이슈_토픽모델링_단어_날짜.xlsx')
company = pd.concat([company1, company2]).reset_index(drop = True, inplace = False)
kospi = pd.read_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/코스피_코스닥_코드/kospi.csv', encoding = 'utf-8')
kospi2 = pd.read_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/코스피_코스닥_코드/코스피.csv', encoding = 'cp949')

In [ ]:
kospi2 = kospi2.drop(['표준코드', '단축코드', '한글 종목명', '영문 종목명', '상장일', '시장구분', '증권구분', '소속부', '주식종류', '액면가', '상장주식수'], axis = 1)
kospi_final = pd.merge(kospi2,kospi, how='left', left_on='한글 종목약명', right_on='Name')
kospi_final.dropna(subset = ["extract"], inplace=True)
kospi_final = kospi_final.reset_index(drop = True, inplace = False)
stocks = fdr.StockListing('KRX')
stocks = stocks.drop(['Sector', 'Industry', 'ListingDate', 'SettleMonth', 'Representative', 'HomePage', 'Region'], axis = 1)
kosdaq = stocks[stocks['Market'] == 'KOSDAQ']
mask = kosdaq['Symbol'].str.len() == 6
kosdaq = kosdaq.loc[mask]
kosdaq = kosdaq.reset_index(drop = True, inplace = False)

In [ ]:
def search_words(text):
  result = re.findall(r'\d+', text)
  return " ".join(result)

kosdaq['extract'] = kosdaq['Symbol'].apply(lambda x: search_words(x))
mask = kosdaq['extract'].str.len() == 6
kosdaq = kosdaq.loc[mask]
kosdaq = kosdaq.reset_index(drop = True, inplace = False)
kospi_final = kospi_final.drop(['한글 종목약명'], axis = 1)
kospi_kosdaq = pd.concat([kospi_final, kosdaq], axis = 0).reset_index(drop = True, inplace = False)
company = pd.merge(company, kospi_kosdaq, left_on = 'company', right_on = 'Name').drop(['Name'], axis = 1).reset_index(drop = True, inplace = False)

In [ ]:
oct_18 = pd.read_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/[종목크롤링]/data_4039_20211115.csv', encoding = 'cp949')
oct_19 = pd.read_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/[종목크롤링]/data_4108_20211115.csv', encoding = 'cp949')
oct_20 = pd.read_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/[종목크롤링]/data_4119_20211115.csv', encoding = 'cp949')
oct_21 = pd.read_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/[종목크롤링]/data_4339_20211115.csv', encoding = 'cp949')
oct_22 = pd.read_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/[종목크롤링]/data_4347_20211115.csv', encoding = 'cp949')
oct_18 = oct_18[['종목코드', '종가']]
oct_19 = oct_19[['종목코드','종가']]
oct_20 = oct_20[['종목코드','종가']]
oct_21 = oct_21[['종목코드','종가']]
oct_22 = oct_22[['종목코드','종가']]
oct_18.columns = ['종목코드', '2021-10-18']
oct_19.columns = ['종목코드', '2021-10-19']
oct_20.columns = ['종목코드', '2021-10-20']
oct_21.columns = ['종목코드', '2021-10-21']
oct_22.columns = ['종목코드', '2021-10-22']
company = pd.merge(company, oct_18, how = 'left', left_on = 'extract', right_on = '종목코드')
company = pd.merge(company, oct_19, how = 'left', left_on = 'extract', right_on = '종목코드')
company = pd.merge(company, oct_20, how = 'left', left_on = 'extract', right_on = '종목코드')
company = pd.merge(company, oct_21, how = 'left', left_on = 'extract', right_on = '종목코드')
company = pd.merge(company, oct_22, how = 'left', left_on = 'extract', right_on = '종목코드')
company = company.drop(['종목코드_x', '종목코드_y', '종목코드'], axis = 1)

In [ ]:
issue['top'] = issue['top'].apply(lambda k: eval(''.join(k)))
issue['relevant'] = issue['relevant'].apply(lambda k: eval(''.join(k)))
issue_date['top'] = issue_date['top'].apply(lambda k: eval(''.join(k)))
issue_date['relevant'] = issue_date['relevant'].apply(lambda k: eval(''.join(k)))
company['content_clean_words'] = company['content_clean_words'].apply(lambda k: eval(''.join(k)))
company['title_clean_words'] = company['title_clean_words'].apply(lambda k: eval(''.join(k)))

In [ ]:
doclist = []
for i in range(len(issue)):
  nouns = ' '.join(issue['relevant'][i])
  doclist.append(nouns)

# 역시 수행 결과를 새로운 칼럼에 저장
issue['relevant_as_sentence'] = doclist

doclist = []
for i in range(len(issue)):
  nouns = ' '.join(issue['top'][i])
  doclist.append(nouns)

# 역시 수행 결과를 새로운 칼럼에 저장
issue['top_as_sentence'] = doclist

## Test 

In [ ]:
issue_date.columns = ['issue_keywords', 'topic', 'relevant', 'top', '2021-10-18', '2021-10-19', '2021-10-20', '2021-10-21', '2021-10-22']

In [ ]:
company = company.dropna()
company = company.reset_index(drop = True, inplace = False)

dflist4 = []
for i in range(len(issue)):
  issue_rel = pd.DataFrame({'line':issue['relevant_as_sentence'][i], 'topic': issue['topic'][i]}, index = [0])
  company_title = pd.DataFrame({'line': company['title_clean_sentence'], 'topic' : company['company']})
  company_title = company_title.dropna()
  issue_title = pd.concat([issue_rel, company_title])
  issue_title = issue_title.reset_index(drop = True, inplace = False)

  tfidf = TfidfVectorizer(min_df = 2)
  tfidf_matrix_issue = tfidf.fit_transform(issue_title['line'])
  indices_title = pd.Series(issue_title.index, index = issue_title['topic']).drop_duplicates()
  cosine_sim_title = cosine_similarity(tfidf_matrix_issue, tfidf_matrix_issue)

  company_content = pd.DataFrame({'line': company['content_clean_sentence'], 'topic' : company['company']})
  company_content = company_content.dropna()
  issue_content = pd.concat([issue_rel, company_content])
  issue_content = issue_content.reset_index(drop = True, inplace = False)

  tfidf_matrix_issue2 = tfidf.fit_transform(issue_content['line'])
  indices_content = pd.Series(issue_content.index, index = issue_content['topic']).drop_duplicates()
  cosine_sim_content = cosine_similarity(tfidf_matrix_issue2, tfidf_matrix_issue2)

  def get_recommendations_title(issue, cosine_sim = cosine_sim_title):

    idx = indices_title[issue]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True)

    sim_scores = sim_scores[1:51]

    company_indices = [i[0] for i in sim_scores]

    df1 = pd.DataFrame({'company' : company['company'].iloc[company_indices], 'title' : company['title_clean_sentence'].iloc[company_indices], 'similarity' : sim_scores})

    return(df1)
  
  def get_recommendations_content(issue, cosine_sim = cosine_sim_content):

    idx = indices_content[issue]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True)

    sim_scores = sim_scores[1:51]

    company_indices = [i[0] for i in sim_scores]

    df1 = pd.DataFrame({'company' : company['company'].iloc[company_indices], 'title' : company['title_clean_sentence'].iloc[company_indices], 'similarity' : sim_scores})

    return(df1)

  df_title1 = get_recommendations_title(issue['topic'][i])
  df_content1 = get_recommendations_content(issue['topic'][i])
  df_total = pd.concat([df_title1, df_content1])
  df_total = df_total.reset_index(drop = True, inplace = False)
  a = []
  b = []

  for i in range(len(df_total)):
    a.append(df_total['similarity'][i][0])
    b.append(df_total['similarity'][i][1])

  df_total['news_num'] = a
  df_total['similarity'] = b

  df_total2 = df_total.groupby(['company'])['similarity'].sum().reset_index().sort_values(by = 'similarity', ascending = False) 
  df_total2 = df_total2.iloc[0:5, :]

  dflist4.append(df_total2)

In [ ]:
dff4 = pd.DataFrame()
for i in range(len(dflist4)):
  df = pd.DataFrame(dflist4[i])
  df = df.reset_index(drop = True, inplace = False)
  dff4 = pd.concat([dff4, df], axis = 1)

In [ ]:
# Bert 준비 과정
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
  '''
    raise SystemError('GPU device not found')
    '''

# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

PATH = "/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/임시/"
model=torch.load(PATH+'bert_sentiment.pt', map_location=torch.device('cpu'))
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

No GPU available, using the CPU instead.


Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
def convert_input_data(sentences):
  # BERT의 토크나이저로 문장을 토큰으로 분리
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  # 입력 토큰의 최대 시퀀스 길이
  MAX_LEN = 128
  # 토큰을 숫자 인덱스로 변환
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  # 어텐션 마스크 초기화
  attention_masks = []
  # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
  # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
  for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)
  # 데이터를 파이토치의 텐서로 변환
  inputs = torch.tensor(input_ids)
  masks = torch.tensor(attention_masks)
  return inputs, masks

def test_sentences(sentences):
  # 평가모드로 변경
  model.eval()
  # 문장을 입력 데이터로 변환
  inputs, masks = convert_input_data(sentences)
  # 데이터를 GPU에 넣음
  b_input_ids = inputs.to(device)
  b_input_mask = masks.to(device)         
  # 그래디언트 계산 안함
  with torch.no_grad():     
      # Forward 수행
      outputs = model(b_input_ids, 
                      token_type_ids=None, 
                      attention_mask=b_input_mask)
  # 로스 구함
  logits = outputs[0]
  # CPU로 데이터 이동
  logits = logits.detach().cpu().numpy()
  return logits

# 최종 결과값
def get_news_sentiment(articles):
  positive_num=[]
  negative_num=[]
  for j in range(0,len(articles["content"])):
    article=articles["content"][articles.index[j]]
    
    str_list=article.split(". ")
    data1=pd.DataFrame({"sentence":str_list})
    data1=data1.loc[data1["sentence"]!=""]
    data1.reset_index(drop=True,inplace=True)
    sentiment=[]
    for sentence in data1["sentence"]:
      logits = test_sentences([sentence])
      sentiment.append(np.argmax(logits))
    # 기사문장별 긍부정 모아놓은 Dataset
    temp=pd.DataFrame({"sentences":data1["sentence"],"sentiment":sentiment})
    
    # 여기서 긍정, 부정 문장 개수 세고
    pos=len(temp.loc[temp["sentiment"]==1])
    neg=len(temp.loc[temp["sentiment"]==0])
    positive_num.append(pos)
    negative_num.append(neg)
  articles["pos_num"]=positive_num
  articles["neg_num"]=negative_num
  #여기서 긍부정 최종판단까지
  temp_list=[]
  for k in range(len(positive_num)):
    temp_list.append(positive_num[k]+negative_num[k])
  ratio=[]
  for p in range(len(positive_num)):
    ratio.append(negative_num[p]/temp_list[p])
  articles["sentiment"]=ratio
  for i in range(len(articles)):
    if articles['sentiment'][articles.index[i]] >= 0.33:
      articles['sentiment'][articles.index[i]] = 0
    elif articles['sentiment'][articles.index[i]] < 0.33:
      articles['sentiment'][articles.index[i]] = 1

  return articles

def find_relevant_news(rc, date):

  test_df = pd.merge(company, rc, on = 'company')
  related_issue_top = issue.loc[issue['topic'] == test_df['issue'][0]]['top']
  related_issue_top = related_issue_top.reset_index(drop = True, inplace = False)
  appe_list = []
  word_list = []
  for i in range(len(test_df)):
    appearance = 0
    w = []
    for j in range(len(related_issue_top[0])):
      for k in range(len(test_df['content_clean_words'][i])):
        if related_issue_top[0][j] == test_df['content_clean_words'][i][k]:
          appearance += 1     
          w.append(test_df['content_clean_words'][i][k])
        else:
          appearance == appearance
          w.append('NA')
    appe_list.append(appearance)
    word_list.append(w)
  test_df['appearance'] = appe_list
  test_df['word_list'] = word_list
  num = []
  for i in range(len(test_df['appearance'])):
    test_df['word_list'][i] = list(dict.fromkeys(test_df['word_list'][i]))
    test_df['word_list'][i] = list(filter(lambda a: a != 'NA', test_df['word_list'][i]))
    num.append(len(test_df['word_list'][i]))
  test_df['appearance'] = num
    
  if any(test_df['appearance'] >= 4):
    rel_news = test_df.loc[test_df['appearance'] >= 4]
    ###########################
    # 여기서부터 감정분석 들어가야 함 #
    ###########################
    # 기사별 긍부정 검사
    rel_news_sentiment=get_news_sentiment(rel_news)
    # 종목별 긍부정 기사 개수 count
    rel_news_sentiment=rel_news_sentiment[["company","sentiment"]]
    # final_article=rel_news_sentiment.groupby(rel_news_sentiment["sentiment"],rel_news_sentiment["company"]).count()
    
    group=rel_news_sentiment.groupby(["company","sentiment"])
    rel_news_sentiment=group.size().reset_index(name='counts')
    final_article=rel_news_sentiment.pivot_table(index=['company'],columns='sentiment',values='counts',fill_value=0)
    final_article = pd.DataFrame(final_article.to_records())

    final_article = final_article.rename(columns={'0.0': 'negative', '1.0': 'positive'})
    print('-'*100)
    display(final_article)
    final = pd.merge(final_article, rc, on = 'company')
    final = final[['company']]
    print('-'*100)
    print('\n')
    print('최종 매핑 종목입니다.')
    display(final)
    
    
  elif all(test_df['appearance'] < 4):
    print('방금 전 종목 중 해당 이슈를 다룬 기사가 없습니다. 따라서 그 종목들은 연관된 종목이 아닙니다.')

def find_issue():
  date = input('원하는 날짜를 입력하세요 (2021-10-18 식으로): ')
  print('\n그날의 이슈 목록입니다. \n')
  print('-'*40)
  print('-'*40)
  print('-'*40)
  issue_that_day = issue_date.loc[issue_date[date] != 'na']
  for i in range(len(issue_that_day[date])):
    print(issue_that_day['topic'][issue_that_day.index[i]])
  print('-'*40)
  print('-'*40)
  print('-'*40)
  issue_input = input('연관 종목을 찾고 싶은 이슈를 입력하세요: ')
  idx = issue.index[issue['topic'] == issue_input].tolist()[0]
  related_company = dff4.iloc[:, (2*idx) : (2*idx + 2)]
  related_company['issue'] = issue_input
  related_company = pd.merge(related_company, company, on = 'company')
  related_company = related_company[['company', date, 'similarity', 'issue']]
  related_company = related_company.drop_duplicates()
  price = input('\n지금 돈이 얼마나 있으십니까?: ')
  if any(related_company['similarity'] >= 1):
    print(f'\n{date}일, 선택하신 이슈 "{issue_input}"와(과) 관련된 종목 중, 갖고 계신 금액 {price}원으로 살 수 있는 주식은 다음과 같습니다\n')
    rc = related_company.loc[(related_company['similarity'] > 1) & (related_company[date] <= int(price))]
    display(rc)
    print('-'*40)
    further = input('\n관련기사의 감정분석을 진행하시겠습니까? (Yes/No): ')
    print('-'*100)
    if further == 'Yes':
      rel_news = find_relevant_news(rc, date)
      display(rel_news)
    elif further == 'No':
      print('\n서비스를 종료합니다.')


  elif all(related_company['similarity'] < 1):
    print('\n선택하신 이슈와 연관된 종목이 없습니다.')

In [ ]:
find_issue()

원하는 날짜를 입력하세요 (2021-10-18 식으로): 2021-10-18

그날의 이슈 목록입니다. 

----------------------------------------
----------------------------------------
----------------------------------------
LG 전기차 배터리 합작 법인
gs 블루 암모니아
kt 플라스틱 저감 최우수 등급
lg 안테나 모듈 공개
skt 코스텔 협약
넷플릭스 망 이용대가 논란
넷플릭스 영화 극장 개봉
누리호 발사
듄
뮤지컬 잭더리퍼
베놈2
부흐빈더 내한
블랙핑크 더 무비 디즈니 플러스 공급
비트코인 etf 상장
신한은행 AI 서비스
와이키키 브라더스
은행권 집단 대출 현황 공유
이터널스
장르만 로맨스
조치원 해문이
죽어도 다시 한번
카카오 브레인 iccv 우승
카카오 페이 상장
컴투스 메타버스 진출
탄소 중립
티빙 라인 일본 협력 진출
----------------------------------------
----------------------------------------
----------------------------------------
연관 종목을 찾고 싶은 이슈를 입력하세요: 탄소 중립

지금 돈이 얼마나 있으십니까?: 100000000

2021-10-18일, 선택하신 이슈 "탄소 중립"와(과) 관련된 종목 중, 갖고 계신 금액 100000000원으로 살 수 있는 주식은 다음과 같습니다



,company,2021-10-18,similarity,issue
0,SK,261000.0,4.001252,탄소 중립
127,POSCO,319500.0,3.389141,탄소 중립
217,그린케미칼,10150.0,2.599932,탄소 중립
221,현대차,209500.0,2.295443,탄소 중립
395,신한지주,39350.0,2.155883,탄소 중립


----------------------------------------

관련기사의 감정분석을 진행하시겠습니까? (Yes/No): Yes
----------------------------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


----------------------------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,company,negative,positive
0,POSCO,2,4
1,SK,0,6
2,그린케미칼,0,3


----------------------------------------------------------------------------------------------------


최종 매핑 종목입니다.


,company
0,POSCO
1,SK
2,그린케미칼


None

In [ ]:
company[company['company'] == '그린케미칼']

,company,title,content,date,title_token_okt,content_token_okt,title_clean_words,content_clean_words,title_clean_sentence,content_clean_sentence,Symbol,Market,extract,2021-10-18,2021-10-19,2021-10-20,2021-10-21,2021-10-22
1233,그린케미칼,2030년 온실가스 40 감축 탄소배출권株 강세 그린케미칼 에코바이오,정부는 오는 2030년까지 온실가스 배출량 40 감축 2050년 탄소중립 시나리...,2021-10-19 12:12:00,"[('2030년', 'Number'), ('온실가스', 'Noun'), ('40',...","[('정부', 'Noun'), ('는', 'Josa'), ('오는', 'Verb')...","[온실가스, 감축, 탄소배출권, 강세, 그린, 케미칼, 에코, 바이오]","[정부, 오는, 온실가스, 배출, 량, 감축, 탄소, 중립, 시나리오, 확정, 매경...",온실가스 감축 탄소배출권 강세 그린 케미칼 에코 바이오,정부 오는 온실가스 배출 량 감축 탄소 중립 시나리오 확정 매경 정부 오는 온실가스...,083420,KOSPI,083420,10150.0,11300.0,11300.0,10700.0,10700.0
1234,그린케미칼,특징주 온실가스 배출량 40 감축에 그린케미칼 등 탄소배출권 관련주 강세,EU 탄소배출권 가격이 사상 최고치를 기록했다. 연합뉴스 정부가 2030년까지 온...,2021-10-19 10:08:00,"[('특징', 'Noun'), ('주', 'Noun'), ('온실가스', 'Noun...","[('EU', 'Alpha'), ('탄소배출권', 'Noun'), ('가격', 'N...","[특징, 온실가스, 배출, 량, 감축, 그린, 케미칼, 탄소배출권, 강세]","[EU, 탄소배출권, 가격, 사상, 최고, 치를, 정부, 온실가스, 배출량, 대비,...",특징 온실가스 배출 량 감축 그린 케미칼 탄소배출권 강세,EU 탄소배출권 가격 사상 최고 치를 정부 온실가스 배출량 대비 감축 목표 확정 초...,083420,KOSPI,083420,10150.0,11300.0,11300.0,10700.0,10700.0
1235,그린케미칼,文 탄소배출 40 감축선언 에 그린케미칼 에코바이오 등 관련주 급...,문재인 대통령이 18일 서울 용산구 노들섬다목적홀에서 열린 2050 탄소중립위원회 ...,2021-10-19 09:57:00,"[('文', 'Foreign'), ('탄소', 'Noun'), ('배출', 'Nou...","[('문재인', 'Noun'), ('대통령', 'Noun'), ('이', 'Josa...","[탄소, 배출, 감축, 선언, 그린, 케미칼, 에코, 바이오, 급]","[대통령, 용산구, 목적, 홀, 열린, 탄소, 중립, 위원회, 전체, 발언, 연합,...",탄소 배출 감축 선언 그린 케미칼 에코 바이오 급,대통령 용산구 목적 홀 열린 탄소 중립 위원회 전체 발언 연합 김용재 정부 오는 온...,083420,KOSPI,083420,10150.0,11300.0,11300.0,10700.0,10700.0
1236,그린케미칼,특징주 온실가스 감축목표 상향 발표에 그린케미칼 상한가,온실가스 감축목표 상향 발표에 그린케미칼이 상한가다. 그린케미칼은 19일 오전 9...,2021-10-19 09:39:00,"[('특징', 'Noun'), ('주', 'Noun'), ('온실가스', 'Noun...","[('온실가스', 'Noun'), ('감축', 'Noun'), ('목표', 'Nou...","[특징, 온실가스, 감축, 목표, 상향, 발표, 그린, 케미칼, 상한]","[온실가스, 감축, 목표, 상향, 발표, 그린, 케미칼, 상한, 가다, 그린, 케미...",특징 온실가스 감축 목표 상향 발표 그린 케미칼 상한,온실가스 감축 목표 상향 발표 그린 케미칼 상한 가다 그린 케미칼 전일 오른 거래 ...,083420,KOSPI,083420,10150.0,11300.0,11300.0,10700.0,10700.0
